# 📊 Задание по методам валидации в машинном обучении  
---  
> урок_№: 1    
домашняя_работа_№: 1  
исполнитель: Маъруф Ибрагимов  

## 1. Загрузка и предобработка данных

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### 📥 **Загрузите набор данных** Bank Customer Churn Prediction и обработайте пропуски.

In [3]:
source_file='Bank Customer Churn Prediction.csv'

In [4]:
df=pd.read_csv(source_file)

In [5]:
df.isna().sum()

customer_id         0
credit_score        0
country             0
gender              0
age                 0
tenure              0
balance             0
products_number     0
credit_card         0
active_member       0
estimated_salary    0
churn               0
dtype: int64

In [6]:
df.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       10000 non-null  int64  
 1   credit_score      10000 non-null  int64  
 2   country           10000 non-null  object 
 3   gender            10000 non-null  object 
 4   age               10000 non-null  int64  
 5   tenure            10000 non-null  int64  
 6   balance           10000 non-null  float64
 7   products_number   10000 non-null  int64  
 8   credit_card       10000 non-null  int64  
 9   active_member     10000 non-null  int64  
 10  estimated_salary  10000 non-null  float64
 11  churn             10000 non-null  int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 937.6+ KB


In [8]:
round(df.describe().T, 2)

,count,mean,std,min,25%,50%,75%,max
customer_id,10000.0,15690940.57,71936.19,15565701.00,15628528.25,15690738.00,15753233.75,15815690.00
credit_score,10000.0,650.53,96.65,350.00,584.00,652.00,718.00,850.00
age,10000.0,38.92,10.49,18.00,32.00,37.00,44.00,92.00
tenure,10000.0,5.01,2.89,0.00,3.00,5.00,7.00,10.00
balance,10000.0,76485.89,62397.41,0.00,0.00,97198.54,127644.24,250898.09
products_number,10000.0,1.53,0.58,1.00,1.00,1.00,2.00,4.00
credit_card,10000.0,0.71,0.46,0.00,0.00,1.00,1.00,1.00
active_member,10000.0,0.52,0.50,0.00,0.00,1.00,1.00,1.00
estimated_salary,10000.0,100090.24,57510.49,11.58,51002.11,100193.92,149388.25,199992.48
churn,10000.0,0.20,0.40,0.00,0.00,0.00,0.00,1.00


In [9]:
df.describe(include='object').T

,count,unique,top,freq
country,10000,3,France,5014
gender,10000,2,Male,5457


In [10]:
df['country'].unique().tolist()

['France', 'Spain', 'Germany']

> Пропусков в файле не обнаружено.

### 🔄 **Разделите данные на признаки (X) и целевую переменную (y).**

In [11]:
df2=df.copy()

In [12]:
target='churn'

X=df2.drop(labels=[target], axis=1)
y=df2[target]

print(X.shape)
print(y.shape)

(10000, 11)
(10000,)


### 📈 **Выведите корреляционную сортированную таблицу** признаков и таргета для определения самых важных признаков, которые влияют на целевую переменную (либо в виде сортированного графика)

In [13]:
corr=df2.select_dtypes(include='number').corr()
target_corr=(
    corr.query("index!=@target")[[target]]
    .assign(abs_corr_coef=lambda d_:d_[target].abs())
    .sort_values(by=['abs_corr_coef'], ascending=False)
)
target_corr

,churn,abs_corr_coef
age,0.285323,0.285323
active_member,-0.156128,0.156128
balance,0.118533,0.118533
products_number,-0.047820,0.047820
credit_score,-0.027094,0.027094
tenure,-0.014001,0.014001
estimated_salary,0.012097,0.012097
credit_card,-0.007138,0.007138
customer_id,-0.006248,0.006248


### 📊 **Выведите распределение целевой переменной в процентном соотношении.**

In [14]:
y.value_counts(normalize=True)

churn
0    0.7963
1    0.2037
Name: proportion, dtype: float64

## 2. Удерживающая проверка (Hold-Out CV)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from catboost import CatBoostClassifier, Pool

### ✂️ **Разделите данные на обучающую и тестовую выборки** в соотношении 80/20.

In [16]:
seed=42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=seed)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8000, 11)
(2000, 11)
(8000,)
(2000,)


### 🧠 **Обучите модель** CatBoostClassifier с использованием обучающей выборки.

In [17]:
df2.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [18]:
{c:df2[c].nunique() for c in df2.columns}

{'customer_id': 10000,
 'credit_score': 460,
 'country': 3,
 'gender': 2,
 'age': 70,
 'tenure': 11,
 'balance': 6382,
 'products_number': 4,
 'credit_card': 2,
 'active_member': 2,
 'estimated_salary': 9999,
 'churn': 2}

In [20]:
df2.groupby('tenure')[target].value_counts(normalize=True).unstack().sort_index()

churn,0,1
tenure,,
0,0.769976,0.230024
1,0.775845,0.224155
2,0.808206,0.191794
3,0.788900,0.211100
4,0.794742,0.205258
5,0.793478,0.206522
6,0.797311,0.202689
7,0.827821,0.172179
8,0.807805,0.192195


In [21]:
cat_features=['country', 'gender', 'products_number', 'credit_card', 'active_member', 'tenure']

In [22]:
train_pool=Pool(data=X_train, label=y_train, cat_features=cat_features)
test_pool=Pool(data=X_test, label=y_test, cat_features=cat_features)

In [23]:
model=CatBoostClassifier(
    random_state=seed,
    depth=2,
    eval_metric='AUC',
    verbose=0
)

In [26]:
model.fit(train_pool, eval_set=test_pool, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### 📊 **Оцените её производительность** на тестовой выборке, используя метрику AUC.

In [72]:
train_recall=metrics.recall_score(y_train, model.predict(X_train))
train_precision=metrics.precision_score(y_train, model.predict(X_train))
train_accuracy=metrics.accuracy_score(y_train, model.predict(X_train))
train_auc=metrics.roc_auc_score(y_train, model.predict(X_train))

test_recall=metrics.recall_score(y_test, model.predict(X_test))
test_precision=metrics.precision_score(y_test, model.predict(X_test))
test_accuracy=metrics.accuracy_score(y_test, model.predict(X_test))
test_auc=metrics.roc_auc_score(y_test, model.predict(X_test))

In [73]:
print(train_recall, test_recall)
print(train_auc, test_auc)

0.5127737226277372 0.5038167938931297
0.7390803792496773 0.7295064056584504


In [74]:
metrics_labels=['recall', 'precision', 'accuracy', 'roc_auc']
cv_comparison=pd.DataFrame({
        'cv_type':['Hold-Out CV']*8, 
        'metric':metrics_labels*2, 
        'stat':['mean']*4+['std']*4,
        'value':[test_recall, test_precision, test_accuracy, test_auc]+[0]*4
})

In [75]:
cv_comparison

,cv_type,metric,stat,value
0,Hold-Out CV,recall,mean,0.503817
1,Hold-Out CV,precision,mean,0.733333
2,Hold-Out CV,accuracy,mean,0.866500
3,Hold-Out CV,roc_auc,mean,0.729506
4,Hold-Out CV,recall,std,0.000000
5,Hold-Out CV,precision,std,0.000000
6,Hold-Out CV,accuracy,std,0.000000
7,Hold-Out CV,roc_auc,std,0.000000


## 3. k-блочная перекрестная проверка (k-Fold CV)

In [76]:
from sklearn.model_selection import cross_val_score

### 🔁 **Реализуйте k-Fold перекрестную проверку** с использованием 5 блоков.

In [77]:
model_2=CatBoostClassifier(
    random_state=seed,
    depth=2,
    iterations=100,
    eval_metric='AUC',
    verbose=0,
    cat_features=cat_features
)

In [80]:
cv_comparison.loc[len(cv_comparison)]=['test', 's', 'd', 2]

### 📊 **Вычислите среднее значение AUC** для всех блоков.

In [86]:
for label in metrics_labels:
    scores = cross_val_score(model_2, X,y, cv=10, scoring=label)
    cv_comparison.loc[len(cv_comparison)]=['k-Fold CV', label, 'mean', scores.mean()]
    cv_comparison.loc[len(cv_comparison)]=['k-Fold CV', label, 'std', scores.std()]

In [87]:
cv_comparison

,cv_type,metric,stat,value
0,Hold-Out CV,recall,mean,0.503817
1,Hold-Out CV,precision,mean,0.733333
2,Hold-Out CV,accuracy,mean,0.866500
3,Hold-Out CV,roc_auc,mean,0.729506
4,Hold-Out CV,recall,std,0.000000
5,Hold-Out CV,precision,std,0.000000
6,Hold-Out CV,accuracy,std,0.000000
7,Hold-Out CV,roc_auc,std,0.000000
8,k-Fold CV,recall,mean,0.464433
9,k-Fold CV,recall,std,0.041854
